In [65]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn import preprocessing
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [86]:
movie_df = pd.read_csv('imdb_top_1000.csv')
movie_df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [87]:
movie_df[['Runtime', 'drop']] = movie_df['Runtime'].str.split(' ', 1, expand=True)

C:\Users\Cole\AppData\Local\Temp\ipykernel_11908\71260551.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  movie_df[['Runtime', 'drop']] = movie_df['Runtime'].str.split(' ', 1, expand=True)


In [88]:
movie_df.drop(columns=['Poster_Link', 'Overview', 'Star1', 'Star2', 'Star3', 'Star4', 'Meta_score', 'Gross', 'Certificate', 'drop'], inplace=True)

In [89]:
movie_df.set_index('Series_Title', inplace=True)

In [90]:
le = LabelEncoder()
le.fit(movie_df.Director.unique())
movie_df['Director'] = le.transform(movie_df.Director)

In [91]:
movie_df['Released_Year'].replace({'PG': 1995}, inplace=True)

In [92]:
movie_df['Released_Year'] = movie_df['Released_Year'].apply(int)
movie_df['Runtime'] = movie_df['Runtime'].apply(int)

In [93]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, The Shawshank Redemption to The 39 Steps
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Released_Year  1000 non-null   int64  
 1   Runtime        1000 non-null   int64  
 2   Genre          1000 non-null   object 
 3   IMDB_Rating    1000 non-null   float64
 4   Director       1000 non-null   int32  
 5   No_of_Votes    1000 non-null   int64  
dtypes: float64(1), int32(1), int64(3), object(1)
memory usage: 50.8+ KB


In [94]:
movie_df

,Released_Year,Runtime,Genre,IMDB_Rating,Director,No_of_Votes
Series_Title,,,,,,
The Shawshank Redemption,1994,142,Drama,9.3,141,2343110
The Godfather,1972,175,"Crime, Drama",9.2,137,1620367
The Dark Knight,2008,152,"Action, Crime, Drama",9.0,83,2303232
The Godfather: Part II,1974,202,"Crime, Drama",9.0,137,1129952
12 Angry Men,1957,96,"Crime, Drama",9.0,456,689845
...,...,...,...,...,...,...
Breakfast at Tiffany's,1961,115,"Comedy, Drama, Romance",7.6,50,166544
Giant,1956,201,"Drama, Western",7.6,164,34075
From Here to Eternity,1953,118,"Drama, Romance, War",7.6,145,43374


In [113]:
genre = 'Drama'
test = []
a = []
for genre in movie_df:
    a.append(movie_df[movie_df['Genre'].apply(lambda x: pd.Series(x.split(', ')).isin([genre]).any())])
    if a == 'Drama':
        test.append(1)
    else:
        test.append(0)

In [114]:
print(a)

[Empty DataFrame
Columns: [Released_Year, Runtime, Genre, IMDB_Rating, Director, No_of_Votes, Outcome]
Index: [], Empty DataFrame
Columns: [Released_Year, Runtime, Genre, IMDB_Rating, Director, No_of_Votes, Outcome]
Index: [], Empty DataFrame
Columns: [Released_Year, Runtime, Genre, IMDB_Rating, Director, No_of_Votes, Outcome]
Index: [], Empty DataFrame
Columns: [Released_Year, Runtime, Genre, IMDB_Rating, Director, No_of_Votes, Outcome]
Index: [], Empty DataFrame
Columns: [Released_Year, Runtime, Genre, IMDB_Rating, Director, No_of_Votes, Outcome]
Index: [], Empty DataFrame
Columns: [Released_Year, Runtime, Genre, IMDB_Rating, Director, No_of_Votes, Outcome]
Index: [], Empty DataFrame
Columns: [Released_Year, Runtime, Genre, IMDB_Rating, Director, No_of_Votes, Outcome]
Index: []]


In [75]:
X = movie_df.drop(columns=['Genre']) 
y = movie_df.Genre

In [76]:
X.head()

,Released_Year,Runtime,IMDB_Rating,Director,No_of_Votes
Series_Title,,,,,
The Shawshank Redemption,1994,142,9.3,141,2343110
The Godfather,1972,175,9.2,137,1620367
The Dark Knight,2008,152,9.0,83,2303232
The Godfather: Part II,1974,202,9.0,137,1129952
12 Angry Men,1957,96,9.0,456,689845


In [77]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)

In [78]:
LogReg = LogisticRegression()

scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

LogReg.fit(X_train,y_train)

y_pred=LogReg.predict(X_test)

In [79]:
print('Classes', LogReg.classes_)
print('Intercept',LogReg.intercept_)
print("Coefficients", LogReg.coef_ )

Classes [  0   1   2   4   6   7   9  10  11  12  13  14  15  16  17  18  19  20
  21  22  23  24  25  26  27  28  30  31  32  33  34  35  36  38  39  40
  41  42  43  44  46  47  48  50  51  52  53  54  55  56  57  58  59  60
  61  62  63  64  66  67  68  69  71  72  74  75  77  78  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  99 100 101 102 103
 104 105 106 107 108 109 110 111 112 113 114 115 116 118 119 120 121 122
 123 124 125 126 127 129 130 132 133 134 135 136 137 138 139 140 141 142
 143 144 145 146 147 149 150 151 152 154 155 156 157 158 160 161 162 163
 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181
 182 184 188 190 191 192 194 195 196 197 198 199 200 201]
Intercept [ 5.49494523e-01 -9.90138842e-01  1.15878330e+00  1.55312864e+00
  6.14058248e-01 -7.12915560e-01 -6.88086275e-01 -1.20423849e+00
  1.42184588e+00  7.49743853e-01 -4.79368053e-01 -2.12268141e-01
 -5.00497131e-01  9.71598601e-01  3.28694335e-01  1.47746442e-01
 -5.281

In [80]:
print("Accuracy", LogReg.score(X_test, y_test))
print(classification_report(y_test, LogReg.predict(X_test)))

Accuracy 0.08
              precision    recall  f1-score   support

           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         1
           4       0.33      0.33      0.33         3
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         1
          11       0.22      0.33      0.27         6
          12       0.00      0.00      0.00         1
          13       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         1
          16       0.00      0.00      0.00         1
          17       0.00      0.00      0.00         2
          22       0.12      0.17      0.14         6
          24       0.00      0.00      0.00         4
          25       0.00      0.00      0.00         2
          26       0.00      0.00      0.00         1
          27       0.00      0.00      0.00         1
          28 

C:\Users\Cole\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Cole\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Cole\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: Undefined